<a href="https://colab.research.google.com/github/kenttw/kent-ai-learning-notebook/blob/master/Machine_Learning_Exercise/CreditCard/SimpleModel_LightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# if you have google drive permission then use this 

from google.colab import drive
drive.mount('/gdrive')
!ln -s   /gdrive/MyDrive/colab/Credit_Card/data/ /content/data

In [2]:
!pip3 install gdown
!mkdir -p ./data/
import gdown


url = 'https://drive.google.com/uc?id=%s'%("1NZ8jMoqcCDpydHxWm6j415HwVS093l4U")
output = './data/X_sample.csv'
gdown.download(url, output, quiet=False)


url = 'https://drive.google.com/uc?id=%s'%("18w2LpyPbydD9qoVQ33GZI2LOjyRY2LaI")
output = './data/y_sample.csv'
gdown.download(url, output, quiet=False)

url = 'https://drive.google.com/uc?id=%s'%("1-0TD5rSfzG3frbgT4fseGBn-Dn5OpGKy")
output = './data/X_test.csv'
gdown.download(url, output, quiet=False)

url = 'https://drive.google.com/uc?id=%s'%("1-1p1dHQVV2JnGP4eMfzJsQwErum_wuMG")
output = './data/y_test.csv'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1NZ8jMoqcCDpydHxWm6j415HwVS093l4U
To: /Users/kentshih/PycharmProjects/kent-ai-learning-notebook/Machine_Learning_Exercise/CreditCard/data/X_sample.csv
48.9MB [00:04, 10.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=18w2LpyPbydD9qoVQ33GZI2LOjyRY2LaI
To: /Users/kentshih/PycharmProjects/kent-ai-learning-notebook/Machine_Learning_Exercise/CreditCard/data/y_sample.csv
100%|██████████| 288k/288k [00:00<00:00, 5.27MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-0TD5rSfzG3frbgT4fseGBn-Dn5OpGKy
To: /Users/kentshih/PycharmProjects/kent-ai-learning-notebook/Machine_Learning_Exercise/CreditCard/data/X_test.csv
609MB [00:57, 10.6MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1-1p1dHQVV2JnGP4eMfzJsQwErum_wuMG
To: /Users/kentshih/PycharmProjects/kent-ai-learning-notebook/Machine_Learning_Exercise/CreditCard/data/y_test.csv
3.59MB [00:00, 10.1MB/s]


'./data/y_test.csv'

In [1]:
import pandas as pd

In [2]:
from sklearn.metrics import roc_auc_score, confusion_matrix, precision_score, recall_score, f1_score

In [3]:
df_X = pd.read_csv("./data/X_sample.csv")
df_X = df_X.set_index("txkey")

In [4]:
df_y = pd.read_csv("./data/y_sample.csv")
df_y = df_y.set_index("txkey")

In [5]:
import lightgbm as lgb


In [6]:
from sklearn.model_selection import train_test_split
df_X_train, df_X_val = train_test_split(df_X, test_size=0.25, random_state=42)


In [7]:
df_y_train = df_y.loc[df_X_train.index]
df_y_val = df_y.loc[df_X_val.index]

In [8]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(df_X_train, df_y_train)
lgb_eval = lgb.Dataset(df_X_val, df_y_val, reference=lgb_train)

In [9]:
# specify your configurations as a dict
params={}
params['learning_rate']=0.03
params['boosting_type']='gbdt' #GradientBoostingDecisionTree
params['objective']='binary' #Binary target feature
params['metric']='binary_logloss' #metric for binary classification
params['max_depth']=10

print('Starting training...')

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=20)



Starting training...
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 11501, number of negative: 11423
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006062 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2188
[LightGBM] [Info] Number of data points in the train set: 22924, number of used features: 176
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501701 -> initscore=0.006805
[LightGBM] [Info] Start training from score 0.006805
[1]	valid_0's binary_logloss: 0.670538
Training until validation scores don't improve for 20 

# Validation with downsampleing data

In [10]:
print('Starting predicting...')
# predict
y_pred = gbm.predict(df_X_val, num_iteration=gbm.best_iteration)
# eval
print('auc:', roc_auc_score(df_y_val, y_pred))

Starting predicting...
auc: 0.9655231831719928


In [11]:
confusion_matrix(df_y_val, (y_pred > 0.5).astype(int))

array([[3543,  317],
       [ 213, 3569]])

In [12]:
tn, fp, fn, tp  = confusion_matrix(df_y_val, (y_pred > 0.5).astype(int)).ravel() 

In [13]:
tn, fp, fn, tp

(3543, 317, 213, 3569)

# Testing : verify with real distribution data

In [11]:
X_test = pd.read_csv("./data/X_test.csv")

In [12]:
X_test = X_test.set_index("txkey")

In [16]:
y_test = pd.read_csv("./data/y_test.csv")

In [17]:
y_test = y_test.set_index("txkey")

In [18]:
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)


In [19]:
y_test.shape

(380447, 1)

In [20]:
print('auc:', roc_auc_score(y_test, y_pred))

auc: 0.9618565375213639


In [21]:
cm = confusion_matrix(y_test, (y_pred > 0.5).astype(int), labels=[0,1])
cm

array([[341759,  33616],
       [   293,   4779]])

In [22]:
th = .6
print("precision_score", precision_score(y_test,(y_pred > th).astype(int)))
print("recall_score",    recall_score(y_test,(y_pred > th).astype(int)))
print("f1_score",        f1_score(y_test,(y_pred > th).astype(int)))

precision_score 0.1468980422069667
recall_score 0.9112776025236593
f1_score 0.2530107291438581


In [23]:
fnscore =  [ (fname,socre) for fname,socre in zip(gbm.feature_name(),gbm.feature_importance())]

In [24]:
top_feature = sorted(fnscore, key=lambda tup: tup[1], reverse=True)[:25]

# Validate with Training Data and Testing Data
* [report](reports/my_report.html)

In [10]:
from evidently.dashboard import Dashboard
from evidently.tabs import DriftTab

In [13]:
data_drift_report = Dashboard(df_X.sample(1000), X_test.sample(1000), tabs = [DriftTab])
data_drift_report.save("reports/my_report.html")
